# Watershed
Segmentación basada en el algoritmo de watershed o "cuenca hidrográfica"

In [1]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
# %matplotlib inline
%matplotlib

import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

Using matplotlib backend: Qt5Agg


In [2]:
# Cargamos la imagen y la binarizamos por el método de Otsu
img = cv.imread('piezas.png')
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)

# Binarizamos...
ret, thresh = cv.threshold(gray,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)

plt.figure()
plt.imshow(cv.cvtColor(img,cv.COLOR_BGR2RGB))

### Marcadores

El método de watershed por OpenCV precisa que se le indiquen los marcadores (semillas) de manera que no se genere sobresegmentacion.

Esto se logra a través de operaciones morfológicas

#### Operaciones morfológicas

Se busca intentar separar fondo de objetos lo mejor posible. Quedará:

- Una zona de fondo seguro
- Una zona de objetos seguro
- Una zona de incertezas (la resta de las dos anteriores)

### Pre-procesamos mediante operaciones morfologicas

In [3]:
# Creamos un elemento estructurante y aplicamos operaciones morfologicas
kernel = np.ones((3,3), np.uint8)

# Aplicamos una serie de iteraciones de apertura
opening = cv.morphologyEx(thresh, cv.MORPH_OPEN, kernel, iterations = 2)

plt.figure()
plt.imshow(thresh, cmap='gray')
plt.title("Img binarizada")

fig=plt.figure('Apertura (reduccion de ruido)')
plt.imshow(opening, cmap='gray')
plt.title("Luego de aplicar una Apertura")

Text(0.5, 1.0, 'Luego de aplicar una Apertura')

### Buscamos las zonas de objeto (sure foreground) y fondo (sure background)

In [4]:
# Área de fondo (background) segura. (Zona negra)
sure_bg = cv.dilate(opening, kernel, iterations=3)

In [5]:
# Búsqueda del área de objeto (foreground) segura.
# Transformación de distancia (distancia de cada píxel al cero más cercano)
#------------------------------------------------------------------------------------
# Conviene cerrar primero la figura para que no haya agujeros (ceros) dentro de los objetos que afecten la transformada
closing = cv.morphologyEx(opening, cv.MORPH_CLOSE, kernel, iterations = 2)
# Ahora sí aplicamos la transformación de distancia
dist_transform = cv.distanceTransform(closing, cv.DIST_L2, maskSize=5)
# Área de objeto (sure foreground) segura. Zona blanca!
ret, sure_fg = cv.threshold(dist_transform, 0.7*dist_transform.max(), 255, 0)
# Búsqueda de la región no definida
sure_fg = np.uint8(sure_fg)
unknown = cv.subtract(sure_bg, sure_fg)

#Graficamos...
#=============
fig=plt.figure('Transformación de distancia')
plt.imshow(dist_transform, cmap='gray')

fig=plt.figure('Zona desconocida')
plt.imshow(unknown, cmap='gray')

fig=plt.figure('Objetos seguros')
plt.imshow(sure_fg, cmap='gray')

#### Etiquetado

En función del análisis morfológico anterior es que se realiza el etiquetado (utilizando componentes conexos)

- Cada supuesto objeto con una etiqueta diferente
- Los pixeles con incertidumbre con etiqueta 0

In [6]:
# Etiquetado de marcadores (similar a flood fill)
ret, connected_coord = cv.connectedComponents(sure_fg)
# Agregar un uno ('1') a todas las etiquetas (para evitar la etiqueta '0')
# CC asigna 0 para el background pero watershed necesita que el 0 este asociado a 
# la region desconocida o indefinida
connected_coord = connected_coord + 1
# Ahora, marcamos la región de incertezas con cero
connected_coord[unknown==255] = 0

fig=plt.figure('Marcadores')
plt.imshow(connected_coord, cmap='jet')
print(connected_coord.shape)

(709, 793)


### Watershed

Una vez definidos los marcadores se corre el algoritmo de watersheed.

Los bordes (devueltos por el algoritmo con valor -1) los "pintamos" de color rojo

In [ ]:
img_out = img.copy()
img_out = cv.cvtColor(img_out, cv.COLOR_BGR2RGB)
# la salida de watershed es una matriz con -1 en las posiciones de bordes y 1 en los pixeles de background
markers = cv.watershed(img_out, connected_coord)
img_out[markers == -1] = [255,0,0]
img_out[markers == 1] = [0,0,0]

fig=plt.figure('Objetos segmentados')
plt.imshow(img_out, cmap='jet')